In [ ]:
import datasets
import models
import tensorflow as tf
import numpy as np
import 

import io
from IPython import display

import matplotlib.pyplot as plt
from models import VGG,Res,Alex


num_classes = 3

def show_image(image):
    plt.imshow(image)
    plt.show()

# Define the features of the dataset
feature_description = {
    'image': tf.io.FixedLenFeature([], tf.string),
    'height': tf.io.FixedLenFeature((), tf.int64),
    'width': tf.io.FixedLenFeature((), tf.int64),
    'channels': tf.io.FixedLenFeature((), tf.int64),
    'label': tf.io.FixedLenFeature([], tf.int64)
}

# Parse each record using the feature description
def parse_example(example_proto):
    example = tf.io.parse_single_example(example_proto, feature_description)
    raw = tf.io.decode_raw(example["image"], out_type=tf.uint8)
    [h,w,c] = [example['height'],example['width'],example['channels']]
    raw = tf.reshape(raw, [h, w, c])
    image = tf.cast(raw/255,tf.float32)
    label = example['label']
    return image, label

tfrecord_filename = './datasets/fruits.tfrecord/'
raw_image_dataset = tf.data.TFRecordDataset(tfrecord_filename)
# Apply the parse function to each record in the dataset
parsed_dataset = raw_image_dataset.map(parse_example)


model = VGG(weights='imagenet', include_top=True)

for image, label in parsed_dataset:
    #print(np.array(image))
    print(image.shape)
    img = np.expand_dims(image,axis=0)
    img = tf.image.resize(img,[224,224])
    predictions = model.predict(img)
    print(predictions)
